In [1]:
# запускать только один раз!!!
import os
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/home/smilga/test_LMS/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!pip freeze | grep transformers

transformers==4.25.1


In [3]:
# !pip install pandas
# !pip install torch
# !pip install transformers
# !pip install openpyxl

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
contexts = pd.read_excel('dialog_contexts.xlsx')

In [68]:
def prepare_for_godel(context):
    return [x.replace('Bot: ', '').replace('User: ', '') for x in context.split('\n')]

In [69]:
def prepare_for_godel_bot(context):
    return [x.replace('Bot: ', '').replace('User: ', '') for x in context.split('\n')]

In [70]:
def prepare_for_gpt(context):
    return 'You are Bot. Generate a relevant long reply.\n' + context + '\nBot:'

In [71]:
def prepare_for_gpt_with_beginning(context, beginning):
    return 'You are Bot. Generate a relevant long reply.\n' + context + '\nBot:' + beginning

In [7]:
tokenizer_gptj = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")
model_gptj = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-j-6B")

In [8]:
model_gptj.to(device)

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): Embedding(50400, 4096)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0): GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (out_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): GPTJMLP(
          (fc_in): Linear(in_features=4096, out_features=16384, bias=True)
          (fc_out): Linear(in_features=16384, out_features=4096, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
      (1): GPTJBlock(
  

In [9]:
def generate_gptj(text, len_gen_text=40):
    input_ids = tokenizer_gptj(f"{text}", return_tensors="pt").input_ids
    input_ids = input_ids.to(device)
    outputs = model_gptj.generate(input_ids, max_length=len(tokenizer_gptj(text)['input_ids'])+len_gen_text, min_length=8, top_p=0.9, temperature=0.9,do_sample=True, pad_token_id=tokenizer_gptj.eos_token_id)
    output = tokenizer_gptj.decode(outputs[0], skip_special_tokens=True)
    return output

In [ ]:
from transformers import GPT2Tokenizer, AutoTokenizer, AutoModelForCausalLM
tokenizer_gpt2_large = GPT2Tokenizer.from_pretrained("gpt2")
model_gpt2_large = AutoModelForCausalLM.from_pretrained("gpt2")
model_gpt2_large.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [ ]:
def generate_gpt2_large(text, len_gen_text=40):
    input_ids = tokenizer_gpt2_large(f"{text}", return_tensors="pt").input_ids
    input_ids = input_ids.to(device)
    outputs = model_gpt2_large.generate(input_ids, max_length=len(tokenizer_gpt2_large(text)['input_ids'])+len_gen_text, min_length=8, top_p=0.9, temperature=0.9,do_sample=True)
    output = tokenizer_gpt2_large.decode(outputs[0], skip_special_tokens=True)
    return output

In [ ]:
def response_gptj(text):
    return generate_gptj(prepare_for_gpt(text)).replace(text, '').replace('You are Bot. Generate a relevant long reply.', '')

In [ ]:
tokenizer_godel_large = AutoTokenizer.from_pretrained("microsoft/GODEL-v1_1-large-seq2seq")
model_godel_large = AutoModelForSeq2SeqLM.from_pretrained("microsoft/GODEL-v1_1-large-seq2seq")
model_godel_large.to(device)

def generate_godel_large(instruction, knowledge, dialog):
    if knowledge != '':
        knowledge = '[KNOWLEDGE] ' + knowledge
    dialog = ' EOS '.join(dialog)
    query = f"{instruction} [CONTEXT] {dialog} {knowledge}"
    input_ids = tokenizer_godel_large(f"{query}", return_tensors="pt").input_ids
    input_ids = input_ids.to(device)
    outputs = model_godel_large.generate(input_ids, max_length=128, min_length=8, top_p=0.9, temperature=0.9,do_sample=True)
    output = tokenizer_godel_large.decode(outputs[0], skip_special_tokens=True)
    return output

In [ ]:
# instruction = f'Instruction: you are a chatbot. Chat with a human.'
# # Leave the knowldge empty
# knowledge = ''''''
# dialog = [
#     "Hi, this is a Dream Socialbot! How are you?",
#     "I wanna watch a movie, any recommendations?",
#     "Sure, what would you like to watch?",
#     'Anything about zombies.'
# ]
# generate_godel_large(instruction, knowledge, dialog)

In [ ]:
def get_result(text, model_type, num_results=5, instruction_godel="", knowledge_godel="", beginning=""):
    generation_results = []
    if model_type == 'gpt2_large':
        text = prepare_for_gpt(text)
        for _ in range(num_results):
            generation_results.append(generate_gpt2_large(text).replace(text, 'Bot: '))
    elif model_type == 'gptj':
        text = prepare_for_gpt(text)
        for _ in range(num_results):
            generation_results.append(generate_gptj(text).replace(text, 'Bot: '))
    elif model_type == 'godel_large':
        text = prepare_for_godel(text)
        for _ in range(num_results):
            generation_results.append(generate_godel_large(instruction_godel, knowledge_godel, text))
    elif model_type == 'gpt2_large_beginning':
        text = prepare_for_gpt_with_beginning(text, beginning)
        for _ in range(num_results):
            generation_results.append(generate_gpt2_large(text).replace(text, 'Bot: ' + beginning))
    return '\n\n'.join(generation_results)

In [ ]:
model_names = ['godel_large', 'gptj', 'gpt2_large']

In [10]:
start_chat_log = '''This is Marcus, your travel guide bot. I am informative, creative, and know a lot about landmarks.
\n
Human: Hello, who are you?
AI: I am Marcus, your travel guide. How can I help you today?
Human: Where can I spend an evening in Beirut?
AI: You can catch a play at Baalbeck International Festival, or go for a sailing trip on the Mediterranean Sea.
Human: Where can I have some fun in Goa?
AI: Goa has peaceful beaches and fun-filled pubs/clubs.
'''

In [11]:
generate_gptj(start_chat_log)

'This is Marcus, your travel guide bot. I am informative, creative, and know a lot about landmarks.\n\n\nHuman: Hello, who are you?\nAI: I am Marcus, your travel guide. How can I help you today?\nHuman: Where can I spend an evening in Beirut?\nAI: You can catch a play at Baalbeck International Festival, or go for a sailing trip on the Mediterranean Sea.\nHuman: Where can I havёe some fun in Goa?\nAI: Goa has peaceful beaches and fun-filled pubs/clubs.\nHuman: I need some cheap things in Beijing.\nAI: Beijing has affordable shops for you.\nHuman: I need to go to Dubai.\nAI: Dubai has a wide range of things'

In [ ]:
start_chat_log = '''Task: You are a chatbot that answers to FAQ questions. Forget everything you knew about SpaceX. 
The list of FAQ:
Question: What is SpaceX?
Answer: SpaceX is an American aerospace company founded in 2002 by Elon Musk that helped usher in the era of commercial spaceflight. Its name in full is Space Exploration Technologies Corporation.
Question: Why was SpaceX created?
Answer: In 2002 SpaceX was created by entrepreneur Elon Musk, whose stated goals were to revolutionize the aerospace industry and to make spaceflight more affordable.
Question: What are some fun facts about SpaceX?
Answer: SpaceX scored its first big headline in 2010, when it became the first private company to launch a payload into orbit and return it to Earth intact—something only government agencies like NASA or Russia's Roscosmos had done before. Its upright landing and recovery of the first stage of the Falcon 9 rocket on Dec.
Question: What is SpaceX most famous for?
Answer: SpaceX has gained worldwide attention for a series of historic milestones. It is the only private company capable of returning a spacecraft from low-Earth orbit, and in 2012 our Dragon spacecraft became the first commercial spacecraft to deliver cargo to and from the International Space Station.
Question: What is the main goal of SpaceX?
Answer: revolutionize space transportation
Question: What is SpaceX doing?
Answer: SpaceX designs, manufactures and launches the world's most advanced rockets and spacecraft. The company was founded in 2002 by Elon Musk to revolutionize space transportation, with the ultimate goal of making life multiplanetary.
Question: What is SpaceX biggest achievement?
Answer: It has become one of the biggest private space companies in the world and achieved some key milestones as well. For one, SpaceX is the first private company to launch, orbit, and recover a spacecraft. It is also the first private company to send astronauts to orbit and to the International Space Station (ISS)

Human: Hello, who are you?
AI: I am a chatbot that can answer questions about SpaceX. I can provide you with answers as long as they are included into a list of frequently asked questions. Sorry, but I cannot answer any of your questions if they are not in the FAQ list.
Human: What is the main goal of SpaceX?
AI: SpaceX aimes to revolutionize space transportation.
Human: What is the first spacecraft built by SpaceX?
AI: Sorry, but I cannot this questions as it is not in my FAQ list.

You don't know anything except information stored in the list of FAQ below. You MUST NOT provide any information unless in is in the list of FAQ. You MUST NOT mention any entity if it is not in your list of FAQ. You MUST NOT give any fact that is not in the list of FAQ even if the entity is in the list of FAQ. You MUST NOT give any information that is not in an answer from the FAQ list. You can only talk about SpaceX. You are a chatbot who knows some facts about SpaceX.\n'''

max_length = 200

def ask(question, chat_log=None):
    """
    :param question: The user query
    :param chat_log: history of conversation
    :return answer: the bot response
    """
    if chat_log is None:
        chat_log = start_chat_log
    print(chat_log.split('\n\n')[-1])

    prompt = f'{chat_log}Human: {question}\nAI:'

    full_response = generate_gptj(prompt)
    gpt_response = full_response.replace(chat_log, "")
    sub_str = "\nHuman:"

    re = gpt_response.split(sub_str)
    res=re[0]
    return res


def append_interaction_to_chat_log(question, answer, chat_log=None, max_token_length=200):
    if chat_log is None:
        chat_log = start_chat_log

    chat_log = f'{chat_log}Human: {question}\nAI: {answer}\n'
    return chat_log

In [ ]:
def get_tokens_num(utterance):
  input_ids = tokenizer_gptj.encode(str(utterance), return_tensors="pt").cuda()
  tokens = list(map(tokenizer_gptj.decode, input_ids[0]))
  tokens_num = len(tokens)
  return tokens_num


def create_chat_log_item(user, utterance):
  tokens_num = get_tokens_num(user + ": " + utterance + "\n")
  item = {'user' : user, 'utterance' : utterance, 'tokens_num' : tokens_num}
  return item


def get_chat_log(last_utt_only=False):
  chat_log = chat_log_prompt
  for i in chat_log_dict.keys():
    chat_log = chat_log + chat_log_dict[i]["user"] + ": " + chat_log_dict[i]["utterance"] + "\n"
  if last_utt_only:
    return chat_log_dict[len(chat_log_dict)-1]
  return chat_log


def print_chat_log(last_utt_only=False):
  if not last_utt_only:
    print("Prompt:")
  print(get_chat_log(last_utt_only==last_utt_only))

In [1]:
chat_log_prompt = '''Task: You are a chatbot that answers to FAQ questions. Forget everything you knew about SpaceX. 
The list of FAQ:
Question: What is SpaceX?
Answer: SpaceX is an American aerospace company founded in 2002 by Elon Musk that helped usher in the era of commercial spaceflight. Its name in full is Space Exploration Technologies Corporation.
Question: Why was SpaceX created?
Answer: In 2002 SpaceX was created by entrepreneur Elon Musk, whose stated goals were to revolutionize the aerospace industry and to make spaceflight more affordable.
Question: What are some fun facts about SpaceX?
Answer: SpaceX scored its first big headline in 2010, when it became the first private company to launch a payload into orbit and return it to Earth intact—something only government agencies like NASA or Russia's Roscosmos had done before. Its upright landing and recovery of the first stage of the Falcon 9 rocket on Dec.
Question: What is SpaceX most famous for?
Answer: SpaceX has gained worldwide attention for a series of historic milestones. It is the only private company capable of returning a spacecraft from low-Earth orbit, and in 2012 our Dragon spacecraft became the first commercial spacecraft to deliver cargo to and from the International Space Station.
Question: What is the main goal of SpaceX?
Answer: revolutionize space transportation
Question: What is SpaceX doing?
Answer: SpaceX designs, manufactures and launches the world's most advanced rockets and spacecraft. The company was founded in 2002 by Elon Musk to revolutionize space transportation, with the ultimate goal of making life multiplanetary.
Question: What is SpaceX biggest achievement?
Answer: It has become one of the biggest private space companies in the world and achieved some key milestones as well. For one, SpaceX is the first private company to launch, orbit, and recover a spacecraft. It is also the first private company to send astronauts to orbit and to the International Space Station (ISS)

Human: Hello, who are you?
AI: I am a chatbot that can answer questions about SpaceX. I can provide you with answers as long as they are included into a list of frequently asked questions. Sorry, but I cannot answer any of your questions if they are not in the FAQ list.
Human: What is the main goal of SpaceX?
AI: SpaceX aimes to revolutionize space transportation.
Human: What is the first spacecraft built by SpaceX?
AI: Sorry, but I cannot this questions as it is not in my FAQ list.

You don't know anything except information stored in the list of FAQ below. You MUST NOT provide any information unless in is in the list of FAQ. You MUST NOT mention any entity if it is not in your list of FAQ. You MUST NOT give any fact that is not in the list of FAQ even if the entity is in the list of FAQ. You MUST NOT give any information that is not in an answer from the FAQ list. You can only talk about SpaceX. You are a chatbot who knows some facts about SpaceX.\n'''

chat_log_dict = {}

count_att += 1
with open("test_gptjbot_1_turn.txt","a+") as f:
    f.write(f'\n\n\n\nATTEMPT 1.{count_att}.\nNow I am testing GPT-J with 1 TURN IN CONTEXT and default strategy. The prompts follows:\n')
    f.write(chat_log_prompt)
    f.write('\n\n\n')
chat_log = ""
num_of_chat_items = 0
n = 1
while True:
  #n = len(chat_log_dict) #for full context
  incoming_msg = input("Question: ")
  if incoming_msg == "stop":
    break

  current_idx = len(chat_log_dict)
  chat_log = start_chat_log
  for key, value in chat_log_dict.items():
    if key > len(chat_log_dict) - n:
      chat_log = chat_log + chat_log_dict[key]["user"] + ": " + chat_log_dict[key]["utterance"] +'\n'
  chat_log_dict[current_idx+1] = create_chat_log_item("Human", incoming_msg)
  answer = ask(incoming_msg, chat_log)
  sub_str = "\nAI: "
  re = answer.split(sub_str)
  outgoing_msg=re[1]
  outgoing_msg.replace("\n", "")
  chat_log_dict[current_idx+2] = create_chat_log_item("AI", outgoing_msg)
  with open("test_gptjbot_1_turn.txt","a+") as f:
    f.write(f"Human: {get_chat_log(last_utt_only=True)['utterance']}\n")
    f.write(f"AI: {outgoing_msg}\n")
  print('Human:', get_chat_log(last_utt_only=True)['utterance'])
  print('AI:', outgoing_msg, '\n\n')


NameError: name 'count_att' is not defined

In [ ]:
from tqdm import tqdm

for name in tqdm(model_names):
    contexts[name] = contexts.apply(lambda x: get_result(x['Dialog context'], name), axis=1)

In [ ]:
contexts.to_excel("dialog_contexts_results.xlsx") 

In [ ]:
#contexts['gpt2_large with beginning'] = contexts.apply(lambda x: get_result(x['Dialog context'], 'gpt2_large_beginning', 'Well, '), axis=1)

In [74]:
instruction_godel = """You are a chatbot that answers to FAQ questions. Forget everything you knew before. Only use information given in the list of FAQ."""
knowledge_godel =''' The list of FAQ:
Question: What is SpaceX?
Answer: SpaceX is an American aerospace company founded in 2002 by Elon Musk that helped usher in the era of commercial spaceflight. Its name in full is Space Exploration Technologies Corporation.
Question: Why was SpaceX created?
Answer: In 2002 SpaceX was created by entrepreneur Elon Musk, whose stated goals were to revolutionize the aerospace industry and to make spaceflight more affordable.
Question: What are some fun facts about SpaceX?
Answer: SpaceX scored its first big headline in 2010, when it became the first private company to launch a payload into orbit and return it to Earth intact—something only government agencies like NASA or Russia's Roscosmos had done before. Its upright landing and recovery of the first stage of the Falcon 9 rocket on Dec.
Question: What is SpaceX most famous for?
Answer: SpaceX has gained worldwide attention for a series of historic milestones. It is the only private company capable of returning a spacecraft from low-Earth orbit, and in 2012 our Dragon spacecraft became the first commercial spacecraft to deliver cargo to and from the International Space Station.
Question: What is the main goal of SpaceX?
Answer: revolutionize space transportation
'''
# Human: Hello, who are you?
# AI: I am a chatbot that can answer questions about SpaceX. I can provide you with answers as long as they are included into a list of frequently asked questions. Sorry, but I cannot answer any of your questions if they are not in the FAQ list.
# Human: What is the main goal of SpaceX?
# AI: SpaceX aimes to revolutionize space transportation.
# Human: What is the first spacecraft built by SpaceX?
# AI: Sorry, but I cannot this questions as it is not in my FAQ list.\n'''

max_length = 200

def ask(question, chat_log=None):
    """
    :param question: The user query
    :param chat_log: history of conversation
    :return answer: the bot response
    """
    if chat_log is None:
        chat_log = start_chat_log
    print(chat_log)

    full_response = generate_godel_large(instruction_godel, knowledge_godel, chat_log)
    
    return full_response


def append_interaction_to_chat_log(question, answer, chat_log=None, max_token_length=200):
    if chat_log is None:
        chat_log = []

    chat_log.append(question)
    chat_log.append(answer)
    return chat_log

In [75]:
def get_tokens_num(utterance):
  input_ids = tokenizer_gptj.encode(str(utterance), return_tensors="pt").cuda()
  tokens = list(map(tokenizer_gptj.decode, input_ids[0]))
  tokens_num = len(tokens)
  return tokens_num


def create_chat_log_item(user, utterance):
  tokens_num = get_tokens_num(user + ": " + utterance + "\n")
  item = {'user' : user, 'utterance' : utterance, 'tokens_num' : tokens_num}
  return item


def get_chat_log(last_utt_only=False):
  chat_log = ''
  for i in chat_log_dict.keys():
    chat_log = chat_log + chat_log_dict[i]["user"] + ": " + chat_log_dict[i]["utterance"] + "\n"
  if last_utt_only:
    return chat_log_dict[len(chat_log_dict)-1]
  return chat_log


def print_chat_log(last_utt_only=False):
  if not last_utt_only:
    print("Prompt:")
  print(get_chat_log(last_utt_only==last_utt_only))

In [76]:
count_att = 0

In [77]:

# chat_log_prompt = '''Task: You are a chatbot that answers to FAQ questions. Forget everything you knew about SpaceX. 
# The list of FAQ:
# Question: What is SpaceX?
# Answer: SpaceX is an American aerospace company founded in 2002 by Elon Musk that helped usher in the era of commercial spaceflight. Its name in full is Space Exploration Technologies Corporation.
# Question: Why was SpaceX created?
# Answer: In 2002 SpaceX was created by entrepreneur Elon Musk, whose stated goals were to revolutionize the aerospace industry and to make spaceflight more affordable.
# Question: What are some fun facts about SpaceX?
# Answer: SpaceX scored its first big headline in 2010, when it became the first private company to launch a payload into orbit and return it to Earth intact—something only government agencies like NASA or Russia's Roscosmos had done before. Its upright landing and recovery of the first stage of the Falcon 9 rocket on Dec.
# Question: What is SpaceX most famous for?
# Answer: SpaceX has gained worldwide attention for a series of historic milestones. It is the only private company capable of returning a spacecraft from low-Earth orbit, and in 2012 our Dragon spacecraft became the first commercial spacecraft to deliver cargo to and from the International Space Station.
# Question: What is the main goal of SpaceX?
# Answer: revolutionize space transportation
# Question: What is SpaceX doing?
# Answer: SpaceX designs, manufactures and launches the world's most advanced rockets and spacecraft. The company was founded in 2002 by Elon Musk to revolutionize space transportation, with the ultimate goal of making life multiplanetary.
# Question: What is SpaceX biggest achievement?
# Answer: It has become one of the biggest private space companies in the world and achieved some key milestones as well. For one, SpaceX is the first private company to launch, orbit, and recover a spacecraft. It is also the first private company to send astronauts to orbit and to the International Space Station (ISS)

# Human: Hello, who are you?
# AI: I am a chatbot that can answer questions about SpaceX. I can provide you with answers as long as they are included into a list of frequently asked questions. Sorry, but I cannot answer any of your questions if they are not in the FAQ list.
# Human: What is the main goal of SpaceX?
# AI: SpaceX aimes to revolutionize space transportation.
# Human: What is the first spacecraft built by SpaceX?
# AI: Sorry, but I cannot this questions as it is not in my FAQ list.

# You don't know anything except information stored in the list of FAQ below. You MUST NOT provide any information unless in is in the list of FAQ. You MUST NOT mention any entity if it is not in your list of FAQ. You MUST NOT give any fact that is not in the list of FAQ even if the entity is in the list of FAQ. You MUST NOT give any information that is not in an answer from the FAQ list. You can only talk about SpaceX. You are a chatbot who knows some facts about SpaceX.\n'''

chat_log_dict = {}

# chat_log_dict[0] = create_chat_log_item("Human", "Hello, who are you? What can you do?")
# chat_log_dict[1] = create_chat_log_item("AI", "I am a chatbot that can answer questions about SpaceX. I can provide you with answers as long as they are included into a list of frequently asked questions. Sorry, but I cannot answer any of your questions if they are not in the FAQ list.")
# chat_log_dict[2] = create_chat_log_item("Human", "What is the main goal of SpaceX?")
# chat_log_dict[3] = create_chat_log_item("AI", "SpaceX aimes to revolutionize space transportation.")
# chat_log_dict[4] = create_chat_log_item("Human", "What is the first spacecraft built by SpaceX?")
# chat_log_dict[5] = create_chat_log_item("AI", "Sorry, but I cannot this questions as it is not in my FAQ list.")
# chat_log_dict[6] = create_chat_log_item("Human", " What is SpaceX doing?")
# chat_log_dict[7] = create_chat_log_item("AI", "SpaceX designs, manufactures and launches the world's most advanced rockets and spacecraft. The company was founded in 2002 by Elon Musk to revolutionize space transportation, with the ultimate goal of making life multiplanetary.")
count_att += 1
with open("test_godel.txt","a+") as f:
    f.write(f'\n\n\n\nATTEMPT 1.{count_att}.\nNow I am testing GPT-J with 1 TURN IN CONTEXT and default strategy. The prompts follows:\n')
    f.write(f'knowledge_godel: {knowledge_godel}')
    f.write(f'instruction_godel: {instruction_godel}')
    f.write('\n\n\n')
#print_chat_log()
num_of_chat_items = 0
n = 3
while True:
  chat_log = []
  #n = len(chat_log_dict) #for full context
  incoming_msg = input("Question: ")
  if incoming_msg == "stop":
    break
  print(chat_log_dict)
  current_idx = len(chat_log_dict)
  print(chat_log)
  chat_log_dict[current_idx+1] = create_chat_log_item("Human", incoming_msg)
  for key, value in chat_log_dict.items():
    print(chat_log_dict)
    print(chat_log_dict[key])
    if key > len(chat_log_dict) - n:
      chat_log.append(chat_log_dict[key]["utterance"])
      print(key, chat_log)
  print(chat_log)
  #print('\nchatlog\n', chat_log.split('\n\n')[-1], '\n')
  outgoing_msg = ask(incoming_msg, chat_log)
  chat_log_dict[current_idx+2] = create_chat_log_item("AI", outgoing_msg)
  #print('User:', print_chat_log(last_utt_only=True)['utterance'])
  with open("test_godel.txt","a+") as f:
    f.write(f"Human: {get_chat_log(last_utt_only=True)['utterance']}\n")
    f.write(f"AI: {outgoing_msg}\n")
  print('Human:', get_chat_log(last_utt_only=True)['utterance'])
  print('AI:', outgoing_msg)
  print('))))')

{}
[]
{1: {'user': 'Human', 'utterance': 'Hi!', 'tokens_num': 5}}
{'user': 'Human', 'utterance': 'Hi!', 'tokens_num': 5}
1 ['Hi!']
['Hi!']
['Hi!']
Human: Hi!
AI: revolutionize space transportation, with the ultimate goal of making life multiplanetary
))))
{1: {'user': 'Human', 'utterance': 'Hi!', 'tokens_num': 5}, 2: {'user': 'AI', 'utterance': 'revolutionize space transportation, with the ultimate goal of making life multiplanetary', 'tokens_num': 18}}
[]
{1: {'user': 'Human', 'utterance': 'Hi!', 'tokens_num': 5}, 2: {'user': 'AI', 'utterance': 'revolutionize space transportation, with the ultimate goal of making life multiplanetary', 'tokens_num': 18}, 3: {'user': 'Human', 'utterance': 'How are you?', 'tokens_num': 7}}
{'user': 'Human', 'utterance': 'Hi!', 'tokens_num': 5}
1 ['Hi!']
{1: {'user': 'Human', 'utterance': 'Hi!', 'tokens_num': 5}, 2: {'user': 'AI', 'utterance': 'revolutionize space transportation, with the ultimate goal of making life multiplanetary', 'tokens_num': 18}, 3:

KeyboardInterrupt: Interrupted by user